In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
!conda install -c conda-forge geocoder --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder# to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India').text

In [3]:
soup = BeautifulSoup(data,'html.parser')
neighborslist=[]
for row in soup.find_all("div",class_="mw-category")[0].findAll('li'):
    neighborslist.append(row.text)
    
hyd_df= pd.DataFrame({'neighborslist':neighborslist})
hyd_df.head()

,neighborslist
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet


In [4]:
hyd_df.shape

(200, 1)

In [5]:
def get_lonlat(neighborslist):
    lat_lon =None
    while (lat_lon is None):
        g= geocoder.arcgis('{}, Hyderabad ,India'.format(neighborslist))
        lat_lon=g.latlng
    return lat_lon

In [6]:
coords = [get_lonlat(neighborslist) for neighborslist in hyd_df['neighborslist']]

In [7]:
coords

[[17.411200000000065, 78.50824000000006],
 [17.392977027745946, 78.45686724051741],
 [17.337650000000053, 78.56414000000007],
 [17.389800000000037, 78.47658000000007],
 [17.410610000000077, 78.51513000000006],
 [17.37751000000003, 78.48005000000006],
 [17.387364823969637, 78.4669870622138],
 [17.34259000000003, 78.47626000000008],
 [17.36068000000006, 78.47998000000007],
 [17.503370000000075, 78.41602000000006],
 [17.535430000000076, 78.54427000000004],
 [17.385820000000024, 78.51836000000003],
 [17.435350000000028, 78.44861000000003],
 [17.40784000000002, 78.49150000000003],
 [17.385140000000035, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.40710000000007, 78.50233000000003],
 [17.372720000000072, 78.49047000000007],
 [17.38897000000003, 78.48681000000005],
 [17.39931000000007, 78.49964000000006],
 [17.339920000000063, 78.54553000000004],
 [17.448510000000056, 78.44924000000003],
 [17.415350000000046, 78.43435000000005],
 [17.38859199570786, 78.47665099785392],
 

In [8]:
df_coords =pd.DataFrame(data=coords,columns=['Latitude','Longitude'])

In [9]:
df_coords.head()

,Latitude,Longitude
0,17.411200,78.508240
1,17.392977,78.456867
2,17.337650,78.564140
3,17.389800,78.476580
4,17.410610,78.515130


In [10]:
hyd_df['Latitude']=df_coords['Latitude']
hyd_df['Longitude']=df_coords['Longitude']
hyd_df.head()

,neighborslist,Latitude,Longitude
0,A. S. Rao Nagar,17.411200,78.508240
1,A.C. Guards,17.392977,78.456867
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adikmet,17.410610,78.515130


In [11]:
hyd_df.to_csv('hyd_data.csv',index=False)

In [12]:
address='Hyderabad,India'

geolocator=Nominatim(user_agent="my application")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('latitude and longitude of hyderabad are {} , {}.'.format(latitude , longitude))

latitude and longitude of hyderabad are 17.38878595 , 78.46106473453146.


In [13]:
map_hyd =folium.Map(location=[latitude,longitude],zoom_start=10)
map_hyd

In [14]:
map_hyd=folium.Map(location=[latitude,longitude],zoom_start=11)

for lat,lng,label in zip(hyd_df["Latitude"],hyd_df["Longitude"],hyd_df["neighborslist"]):
    label='{}'.format("neighborslist")
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],popup=label,radius=5,color='blue',fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_hyd)
    
map_hyd

In [15]:
map_hyd.save('map_hyd.html')

In [16]:
# define Foursquare Credentials and Version
CLIENT_ID = '3MEYXBIUMTDSU3TJJ4PKFI1UDXRM12PVFPLJDXKFDE2STHUE' # your Foursquare ID
CLIENT_SECRET = 'KGHVOF5ZLHXF4ANQWZPXAJXSXJJYBXLBQJSN2JWD1AFJROKK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3MEYXBIUMTDSU3TJJ4PKFI1UDXRM12PVFPLJDXKFDE2STHUE
CLIENT_SECRET:KGHVOF5ZLHXF4ANQWZPXAJXSXJJYBXLBQJSN2JWD1AFJROKK


In [17]:
LIMIT = 100
radius = 2000

venues = []

for lat, long, neighborhood in zip(hyd_df['Latitude'], hyd_df['Longitude'], hyd_df['neighborslist']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
    

In [18]:
venues_df = pd.DataFrame(venues,columns=['Neighborhood','Latitude','Longitude','venuename','venueLatitude','venueLongitude','venuecategory'])
venues_df.head()

,Neighborhood,Latitude,Longitude,venuename,venueLatitude,venueLongitude,venuecategory
0,A. S. Rao Nagar,17.4112,78.50824,Bawarchi,17.406369,78.497662,Indian Restaurant
1,A. S. Rao Nagar,17.4112,78.50824,Sudharshan Theatre 35mm,17.406530,78.495150,Movie Theater
2,A. S. Rao Nagar,17.4112,78.50824,Subway,17.404173,78.514950,Sandwich Place
3,A. S. Rao Nagar,17.4112,78.50824,Devi 70 MM,17.406329,78.495409,Movie Theater
4,A. S. Rao Nagar,17.4112,78.50824,Spencer's,17.412592,78.498400,Convenience Store


In [19]:
venues_df['venuecategory'].unique()

array(['Indian Restaurant', 'Movie Theater', 'Sandwich Place',
       'Convenience Store', 'Ice Cream Shop', 'Coffee Shop', 'Café',
       'Asian Restaurant', 'Bakery', 'Flea Market', 'Shopping Mall',
       'Food', 'Hyderabadi Restaurant', 'Juice Bar', 'Lounge',
       'South Indian Restaurant', 'Park', 'Bistro', 'Science Museum',
       'Snack Place', 'Middle Eastern Restaurant',
       'Vegetarian / Vegan Restaurant', 'Stadium',
       'Performing Arts Venue', 'Hotel', 'Hotel Bar',
       'Fast Food Restaurant', 'Pizza Place', 'Mobile Phone Shop',
       'Fried Chicken Joint', 'Department Store', 'Hookah Bar',
       'Electronics Store', 'Clothing Store', 'Chinese Restaurant',
       'Breakfast Spot', 'Farmers Market', 'Music Store', 'Restaurant',
       'Shoe Store', 'Food Truck', 'Neighborhood', 'Chaat Place', 'Diner',
       'Burger Joint', 'Dessert Shop', 'Smoke Shop', 'Bar', 'Multiplex',
       'Gaming Cafe', 'Indie Movie Theater', 'Food Court', 'Bus Station',
       'Tea Room'

In [20]:
venues_df.groupby('Neighborhood').count()

,Latitude,Longitude,venuename,venueLatitude,venueLongitude,venuecategory
Neighborhood,,,,,,
A. S. Rao Nagar,22,22,22,22,22,22
A.C. Guards,53,53,53,53,53,53
Abhyudaya Nagar,9,9,9,9,9,9
Abids,82,82,82,82,82,82
Adikmet,19,19,19,19,19,19
Afzal Gunj,46,46,46,46,46,46
Aghapura,53,53,53,53,53,53
"Aliabad, Hyderabad",10,10,10,10,10,10
Alijah Kotla,17,17,17,17,17,17


In [21]:
'Shopping Mall' in venues_df['venuecategory'].unique()

True

In [27]:
hyd_shop = pd.get_dummies(venues_df['venuecategory'],prefix="",prefix_sep='')
hyd_shop['Neighborhoods']=venues_df['Neighborhood']
fixed_columns = [hyd_shop.columns[-1]] + list(hyd_shop.columns[:-1])
hyd_shop = hyd_shop[fixed_columns]

print(hyd_shop.shape)
hyd_shop.head()

(6777, 171)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Rajasthani Restaurant,Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
hyd_grouped = hyd_shop.groupby('Neighborhoods').mean().reset_index()
print(hyd_grouped.shape)
hyd_grouped.head()

(200, 171)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Rajasthani Restaurant,Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.045455,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.045455,0.000000,0.000000,0.0,0.000000,0.0,0.090909,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.045455,0.0,0.045455,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.045455,0.318182,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.136364,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045455,0.0,0.000000,0.000000,0.0,0.045455,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,A.C. Guards,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.094340,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.018868,0.0,0.0,0.0,0.018868,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.056604,0.000000,0.018868,0.0,0.018868,0.0,0.018868,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.018868,0.000000,0.000000,0.0,0.0,0.0,0.018868,0.0,0.0,0.0,0.018868,0.037736,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.018868,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018868,0.0,0.075472,0.018868,0.0,0.0,0.037736,0.037736,0.132075,0.0,0.000000,0.0,0.0,0.0,0.0,0.018868,0.0,0.0,0.0,0.0,0.018868,0.0,0.0,0.0,0.0,0.0,0.0,0.018868,0.037

In [32]:
len(hyd_grouped[hyd_grouped['Shopping Mall']>0])

67

In [33]:
hyd =hyd_grouped[['Neighborhoods','Shopping Mall']]
hyd.head()

,Neighborhoods,Shopping Mall
0,A. S. Rao Nagar,0.045455
1,A.C. Guards,0.000000
2,Abhyudaya Nagar,0.000000
3,Abids,0.012195
4,Adikmet,0.000000


In [34]:
k =3
hyd_clust=hyd_grouped.drop(["Neighborhoods"],axis=1)
kmeans =KMeans(n_clusters=k,init='k-means++',random_state=0).fit(hyd_clust)

In [35]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 2, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0])

In [36]:
hyd_copy =hyd.copy()
hyd_copy['cluster labels']= kmeans.labels_
hyd_copy.rename(columns={"Neighborhoods":"Neighborhood"},inplace=True)
hyd_copy.head()

,Neighborhood,Shopping Mall,cluster labels
0,A. S. Rao Nagar,0.045455,1
1,A.C. Guards,0.000000,1
2,Abhyudaya Nagar,0.000000,1
3,Abids,0.012195,1
4,Adikmet,0.000000,1


In [37]:
hyd_copy['Latitude']=hyd_df['Latitude']
hyd_copy['Longitude']= hyd_df['Longitude']
hyd_copy.shape

(200, 5)

In [38]:
hyd_copy.sort_values(["cluster labels"], inplace=True)
hyd_copy.head()

,Neighborhood,Shopping Mall,cluster labels,Latitude,Longitude
199,Sitaphalmandi,0.0,0,17.42800,78.51700
138,Mylargadda,0.0,0,17.42839,78.51606
139,Nacharam,0.0,0,17.43351,78.56673
72,IDA Bollaram,0.0,0,17.55403,78.34459
140,"Nagaram, Medchal–Malkajgiri district",0.0,0,17.60993,78.49122


In [40]:
hyd_copy['cluster labels'].value_counts()

1    140
0     54
2      6
Name: cluster labels, dtype: int64

In [43]:
map_clusters= folium.Map(location=[latitude,longitude],zoom_start=11)
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(hyd_copy['Latitude'],hyd_copy['Longitude'], hyd_copy['Neighborhood'], hyd_copy['cluster labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
map_clusters.save('map_clusters.html')

In [45]:
hyd_copy.loc[hyd_copy["cluster labels"]==0]

,Neighborhood,Shopping Mall,cluster labels,Latitude,Longitude
199,Sitaphalmandi,0.000000,0,17.428000,78.517000
138,Mylargadda,0.000000,0,17.428390,78.516060
139,Nacharam,0.000000,0,17.433510,78.566730
72,IDA Bollaram,0.000000,0,17.554030,78.344590
140,"Nagaram, Medchal–Malkajgiri district",0.000000,0,17.609930,78.491220
142,Nalgonda 'X' Roads,0.000000,0,17.375388,78.498487
146,Nanakramguda,0.000000,0,17.413990,78.345800
59,Gaddiannaram,0.066667,0,17.365000,78.526180
152,Nizampet,0.000000,0,17.518330,78.381860
57,Film Nagar,0.000000,0,17.413910,78.405470


# cluster 2

In [46]:
hyd_copy.loc[hyd_copy["cluster labels"]==1]

,Neighborhood,Shopping Mall,cluster labels,Latitude,Longitude
127,Mettuguda,0.000000,1,17.427740,78.528920
187,Santoshnagar,0.000000,1,17.392100,78.437650
131,Moazzam Jahi Market,0.018182,1,17.384480,78.474420
188,Saroornagar,0.058824,1,17.354420,78.539210
186,Sanjeeva Reddy Nagar,0.000000,1,17.444380,78.447240
185,Sanghi Nagar,0.010000,1,17.402034,78.457069
128,"Minister Road, Hyderabad",0.000000,1,17.432743,78.484542
126,Mehdipatnam,0.000000,1,17.392630,78.442190
190,Serilingampally,0.000000,1,17.482160,78.323000
124,Masab Tank,0.010000,1,17.400930,78.453620


In [47]:
hyd_copy.loc[hyd_copy["cluster labels"]==2]

,Neighborhood,Shopping Mall,cluster labels,Latitude,Longitude
69,Hayathnagar,0.0,2,17.404860,78.481210
119,Mallapur,0.0,2,17.450017,78.609361
159,Patancheru,0.0,2,17.526770,78.252340
130,Miyapur,0.0,2,17.421020,78.582440
77,Jeedimetla,0.0,2,17.521830,78.454330
42,Cherlapally,0.0,2,17.466480,78.599990


# observation

### From the above clusters we can clearly understand that hyderabad has most numbers of shopping malls but  most of shopping malls are centrally located .If we want to open a new shoppping mall ,the best place is in cluster 3, as in cluster 3 we can see there are least number of shopping malls ,whereas in cluster 1 has most number of malls . cluster 2 has moderate number of malls and neighbors with respect to malls. so cluster 3 is the best place to open a new shopping mall.